###5.1 계산 그래프

5.1.1 계산 그래프로 풀다

순전파(forward propagation) : 계산을 왼쪽에서 오른쪽으로 진행

역전파(backwward propagation) : 계산을 오른쪽에서 왼쪽으로 진행

###5.2 연쇄법칙

5.2.1 계산그래프의 역전파

국소적미분 : y=f(x)의 함수의 미분값을 구하는 것

연쇄법칙 : 합성함수의 미분은 합성함수를 구성하는 각 함수의 미분의 곱으로 나타낼 수 있다.

  여러 계산을 전달하여 결국 최종 미분값을 얻음

### 5.3 역전파

5.3.1 덧셈노드의 역전파 : 입력값 그대로 출력

5.3.2 곱셈노드의 역전파: 입력값과 해당값을 곱해서 출력

### 5.4 단순한 계층 구현하기

5.4.1 곱셈 계층

In [2]:
class MulLayer:
    """변수 x, y 초기화: 곱셈계층은 미분값이 차수이므로 초기화 필요
            덧셈은 어차피 미분값이 0이므로 필요X"""
    def __init__(self): 
        self.x = None
        self.y = None

    def forward(self, x, y):
        self.x = x
        self.y = y                
        out = x * y

        return out

    def backward(self, dout):
        dx = dout * self.y  # 상류에서 넘어온 미분(dout)에 순전파 때의 값을 서로 바꿔 곱한 후 출력
        dy = dout * self.x

        return dx, dy

In [3]:
#순전파
apple = 100
apple_num = 2
tax = 1.1

#계층들
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

#순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price) # 220

220.00000000000003


In [6]:
# backward
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)  
dapple, dapple_num = mul_apple_layer.backward(dapple_price)  

print("price:", int(price))
print("dapple:", dapple)
print("dapple_num:", int(dapple_num))
print("dtax:", dtax)

price: 220
dapple: 2.2
dapple_num: 110
dtax: 200


5.4.2 덧셈 계층

In [7]:
class AddLayer:
    def __init__(self):
        pass

    def forward(self, x, y):
        out = x + y

        return out

    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1

        return dx, dy

In [9]:
# coding: utf-8

apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# layer
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)  # (1) 사과가격 구하기
orange_price = mul_orange_layer.forward(orange, orange_num)  # (2) 오렌지 가격 구하기
all_price = add_apple_orange_layer.forward(apple_price, orange_price)  # (3) 사과 가격 + 오렌지 가격
price = mul_tax_layer.forward(all_price, tax)  # (4) 세금 더해진 값 구하기

# backward
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)  # (4) 전체 가격 구하는 것으로 역전파
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)  # (3) 사과, 오렌지 가격 구하는 것으로 역전파
dorange, dorange_num = mul_orange_layer.backward(dorange_price)  # (2) 오렌지 가격, 오렌지 수 구하는 것으로 역전파
dapple, dapple_num = mul_apple_layer.backward(dapple_price)  # (1) 사과 가격, 사과 수 구하는 것으로 역전파

print("price:", int(price))
print("dApple:", dapple)
print("dApple_num:", int(dapple_num))
print("dOrange:", dorange)
print("dOrange_num:", int(dorange_num))
print("dTax:", dtax)

price: 715
dApple: 2.2
dApple_num: 110
dOrange: 3.3000000000000003
dOrange_num: 165
dTax: 650


###5.5 활성화함수 계층 구현하기 

5.5.1 ReLU 계층

In [14]:
import numpy as np

class Relu:
  def __init__(self):
      self.mask = None
    
  def forward(self, x):
      self.mask = (x <= 0)
      out = x.copy()
      out[self.mask] = 0
        
      return out
  
  def backward(self, dout):
      dout[self.mask] = 0
      dx = dout
      
      return dx

In [16]:
x = np.array([[1.0, -0.5], [-2.0, 3.0]])
print(x)
mask = (x <= 0)
print(mask)

[[ 1.  -0.5]
 [-2.   3. ]]
[[False  True]
 [ True False]]
